Nonconjugate model for a bioassay experiment (BDA3 p.74)

In [9]:
import numpy as np
import pandas as pd
from scipy.special import expit  
from scipy.stats import multivariate_normal
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# load data
df = pd.read_csv('../data/bioassay.csv')
x = df.dose.values
n = df.num_animals.values
y = df.num_deaths.values

Maximum likelihood estimate of ($\alpha, \beta$)

In [14]:
# structure data in individual observation format for fitting logistic regression
# it is feasible to use glm in R for fitting a count table. However, it's not possible for LogisticRegression in sklearn
xx = np.repeat(x,n).reshape(-1,1)
yy = [[1]*y[i] + [0]*(n[i]-y[i]) for i in range(len(y))]
yy = np.asarray(yy).ravel()
lm = LogisticRegression(penalty='none', solver='lbfgs').fit(xx,yy)

# M.L.E point estimate
alpha_hat, beta_hat = np.asscalar(lm.intercept_), np.asscalar(lm.coef_)

# Design matrix -- add column of 1's at the beginning of X matrix
X_design = np.hstack([np.ones((xx.shape[0], 1)), xx])
# Initiate matrix of 0's, fill diagonal with each predicted observation's variance
p_hat = lm.predict_proba(xx)
V = np.diagflat(np.product(p_hat, axis=1))

# Covariance matrix
# Note that the @-operater does matrix multiplication in Python 3.5+, so if you're running
# Python 3.5+, you can replace the covLogit-line below with the more readable:
# cov_logit = np.linalg.inv(X_design.T @ V @ X_design)
cov_logit = np.linalg.inv(np.dot(np.dot(X_design.T, V), X_design))
# print("Covariance matrix: ", cov_logit)

# Standard errors
alpha_se, beta_se = np.sqrt(np.diag(cov_logit))
# Wald statistic (coefficient / s.e.) ^ 2
alpha_w, beta_w = alpha_hat / alpha_se**2, beta_hat / beta_se**2

mle_df = pd.DataFrame({'point_estimate':[alpha_hat, beta_hat]
                       , 'standard_error':[alpha_se, beta_se]
                       , 'Wald_statistics': [alpha_w, beta_w]}
                     , index=['alpha','beta'])
print(mle_df)

       point_estimate  standard_error  Wald_statistics
alpha        0.846554        1.019077         0.815156
beta         7.748719        4.872704         0.326355


In [ ]:
# compute the posterior density in grid
A = np.linspace(-4, 8, 100)
B = np.linspace(-10, 40, 100)
ilogit_abx = 1 / (np.exp(-(A[:,None] + B[:,None,None] * x)) + 1)
p = np.prod(ilogit_abx**y * (1 - ilogit_abx)**(n - y), axis=2)